In [146]:
import pandas as pd
import geopandas as gpd
from shapely import wkt, Polygon

city_name = "Dresden"

city = pd.read_csv(f"../input/{city_name}/results.csv")
city['geometry'] = city['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(city, crs='epsg:25832')
gdf

,Unnamed: 0,id,geometry,centroid,node,building,score
0,0,5651,"POLYGON ((832593.267 5665561.344, 832620.962 5...",POINT (832651.3219950458 5665488.143941828),324311126,yes,0.466884
1,1,5758,"POLYGON ((832165.412 5666350.488, 832171.162 5...",POINT (832186.1185630546 5666321.843464637),2544975287,yes,0.454622
2,2,5759,"POLYGON ((832271.652 5666153.940, 832249.088 5...",POINT (832257.8332625618 5666167.364710427),1864886872,hotel,0.485546
3,3,5761,"POLYGON ((832336.382 5666383.377, 832331.898 5...",POINT (832335.2514454264 5666408.18403234),26889715,yes,0.417645
4,4,5781,"POLYGON ((832035.725 5665528.081, 832033.478 5...",POINT (832079.1088381336 5665532.265407829),29283190,apartments,0.507577
...,...,...,...,...,...,...,...
69476,69476,1351800472,"POLYGON ((839316.218 5663657.100, 839290.935 5...",POINT (839302.1758864798 5663649.568013117),2203616096,yes,0.425630
69477,69477,1351800473,"POLYGON ((839273.974 5663637.836, 839283.318 5...",POINT (839281.8325269073 5663637.330463486),2203616096,yes,0.499729
69478,69478,1351999280,"POLYGON ((825714.206 5669896.695, 825719.665 5...",POINT (825713.6297009723 5669890.64430275),9714618954,yes,0.128184
69479,69479,1352072580,"POLYGON ((844055.771 5661329.403, 844062.338 5...",POINT (844057.7664838051 5661325.106818343),6447459730,yes,0.000000


In [147]:
centroids = gdf.centroid
center_x = centroids.x.mean()
center_y = centroids.y.mean()



from shapely.geometry import Point
center = Point(center_x, center_y)

In [148]:
cityhalls = pd.read_csv("../input/cityhalls.csv", index_col = 0)

from pyproj import Transformer

if city_name == "München":
    city_name_hall = "Muenchen"
elif city_name == "Münster":
    city_name_hall = "Muenster"
else:
    city_name_hall = city_name


transformer = Transformer.from_crs("WGS84", "EPSG:25832")
center = Point(transformer.transform(cityhalls.loc[city_name_hall].lat, cityhalls.loc[city_name_hall].lon))

In [149]:
len(city.index)

69481

In [150]:
city_best90 = city.nlargest(round(0.9*len(city.index)), "score", keep = "all")
    
city_best90.to_csv(f"../input/{city_name}/best90.csv")
len(city_best90.index)

62535

In [151]:
city_best90.score.mean()

0.3651017558609142

In [152]:
city_best90.score.std()

0.16714443168326962

In [153]:
city_best90.score.median()

0.3782522001597166

In [154]:
from shapely import distance

point = centroids[0]

distances = centroids.apply(distance, args = (center,))

city.insert(6, "distance_to_middle", distances)
city.to_csv(f"../input/{city_name}/100percent.csv")

In [155]:
city_core = city[city.distance_to_middle < 2000]
len(city_core.index)

2327

In [156]:
city_core.score.mean()

0.4413353678631044

In [157]:
city_core.score.std()

0.13098928956433906

In [158]:
city_core.score.median()

0.4667443896813675

In [159]:
city.score.mean()

0.3289708033760211

In [160]:
city.score.std()

0.19209774894442536

In [161]:
subset_keep = 0.8

number_keep = len(city) * subset_keep
number_keep = round(number_keep)
city_center = city.nsmallest(n = number_keep, columns = ["distance_to_middle"], keep = "all")

city_center = gpd.GeoDataFrame(city_center, crs='epsg:25832')
city_center.to_csv(f"../input/{city_name}/80percent.csv")

city_center.score.mean()

0.3847121967200489

In [162]:
city_keep80best90 = city_center.nlargest(round(0.9*len(city_center.index)), "score", keep = "all")
    
city_keep80best90.to_csv(f"../input/{city_name}/best90keep80.csv")
len(city_keep80best90.index)

50029

In [163]:
city_keep80best90.score.mean()

0.422578964330531

In [164]:
city_keep80best90.score.std()

0.1268908982341365

In [165]:
city_keep80best90.score.median()

0.4315806091020356

In [166]:
city_center.score.std()

0.16619260070925979

In [167]:
subset_keep = 0.6

number_keep = len(city) * subset_keep
number_keep = round(number_keep)
city_center = city.nsmallest(n = number_keep, columns = ["distance_to_middle"], keep = "all")

city_center = gpd.GeoDataFrame(city_center, crs='epsg:25832')
city_center.to_csv(f"../input/{city_name}/60percent.csv")

city_center.score.mean()

0.4241646027080365

In [168]:
city_center.score.std()

0.14228198707180467

In [169]:
subset_keep = 0.5

number_keep = len(city) * subset_keep
number_keep = round(number_keep)
city_center = city.nsmallest(n = number_keep, columns = ["distance_to_middle"], keep = "all")

city_center = gpd.GeoDataFrame(city_center, crs='epsg:25832')
city_center.to_csv(f"../input/{city_name}/50percent.csv")

city_center.score.mean()

0.4404419930325924

In [170]:
city_center.score.std()

0.1361726206214362

In [171]:
p = Polygon([ [ 6.07653, 50.7535 ], [ 6.07286, 50.75483 ], [ 6.07997, 50.76308 ], [ 6.07706, 50.76672 ], [ 6.06919, 50.76122 ], [ 6.06403, 50.76011 ], [ 6.05842, 50.75889 ], [ 6.06056, 50.76208 ], [ 6.05403, 50.76619 ], [ 6.05319, 50.76825 ], [ 6.05256, 50.76983 ], [ 6.05322, 50.77031 ], [ 6.05467, 50.77131 ], [ 6.06258, 50.77675 ], [ 6.061, 50.79 ], [ 6.06208, 50.79061 ], [ 6.06817, 50.79414 ], [ 6.07397, 50.79142 ], [ 6.07606, 50.79161 ], [ 6.079, 50.79189 ], [ 6.08622, 50.78961 ], [ 6.09336, 50.79558 ], [ 6.09706, 50.79467 ], [ 6.10058, 50.79378 ], [ 6.1035, 50.7915 ], [ 6.10783, 50.79014 ], [ 6.11783, 50.79794 ], [ 6.12236, 50.79547 ], [ 6.12364, 50.79478 ], [ 6.12008, 50.78972 ], [ 6.12628, 50.78603 ], [ 6.13381, 50.78153 ], [ 6.15033, 50.78708 ], [ 6.15081, 50.78711 ], [ 6.16031, 50.78753 ], [ 6.16442, 50.78792 ], [ 6.16614, 50.78808 ], [ 6.16694, 50.78208 ], [ 6.16119, 50.77886 ], [ 6.14753, 50.77883 ], [ 6.1425, 50.77425 ], [ 6.15047, 50.7697 ], [ 6.15119, 50.76694 ], [ 6.14692, 50.76328 ], [ 6.14836, 50.76192 ], [ 6.14428, 50.75592 ], [ 6.14339, 50.75458 ], [ 6.14194, 50.75411 ], [ 6.14192, 50.75686 ], [ 6.13325, 50.76003 ], [ 6.13333, 50.76122 ], [ 6.13392, 50.7692 ], [ 6.12958, 50.77192 ], [ 6.12306, 50.76619 ], [ 6.116, 50.76 ], [ 6.11744, 50.75678 ], [ 6.11603, 50.75586 ], [ 6.10808, 50.75814 ], [ 6.10739, 50.75494 ], [ 6.10378, 50.75719 ], [ 6.09731, 50.75581 ], [ 6.09306, 50.74847 ], [ 6.09381, 50.74483 ], [ 6.0895, 50.74436 ], [ 6.08436, 50.75439 ], [ 6.08414, 50.75442 ], [ 6.08075, 50.75486 ], [ 6.07789, 50.753 ], [ 6.07653, 50.7535 ] ])

p_series = gpd.GeoSeries(p, crs='wgs84')
gdf = gdf.to_crs('wgs84')
city_in_polygon = gdf[gdf.within(p)]
city_in_polygon = city_in_polygon.to_crs("epsg:25832")
city_in_polygon.to_csv(f"../input/{city_name}/polygonpercent.csv")


In [172]:
city_center.score.mean()

0.4404419930325924

In [173]:
city_center.score.std()

0.1361726206214362

In [174]:
import pandas as pd
import geopandas as gpd
import folium

EXPORT_PATH = "results"

def create_building_visualisation(buildings:gpd.GeoDataFrame):
    buildings_for_vis = buildings[["id", "node", "building", "score", "geometry", "centroid"]]
    buildings_for_vis.rename(columns = {"id": "OSM ID",
                                        "node": "Zugehöriger Knoten",
                                        "building": "Gebäudetyp",
                                        "score": "Score"},
                             inplace = True)
    buildings_vis = folium.Map(tiles = "CartoDB positron")
    buildings_vis = buildings_for_vis.explore(column = "Score",
                                      cmap = "viridis",
                                      vmin = 0, 
                                      vmax = 1,
                                      m = buildings_vis)
    buildings_vis.save(f"{EXPORT_PATH}/buildings.html")